# Lecture 11: Logistic Regression and Classification

In classification problem, the response variables $y$ are discrete, representing different catagories. For simplicity, we will first introduce the **binary classification case** -- $y$ has only two categories, denoted as $0$ and $1$.

## Model-setup

**Assumption 1**: Dependent on the variable $x$, the response variable $y$ has different **probabilities** to take value in 0 or 1. Instead of predicting exact value of 0 or 1, we are actually predicting the **probabilities**.

**Assumption 2**: Logistic function assumption.

$$P(y=1|\mathbf{x})=f(\mathbf{x};\mathbf{\beta}) = \frac{1}{1 + \exp(-\tilde{x}\mathbf{\beta})}
=: \sigma(\tilde{x}\mathbf{\beta}). $$

where $\sigma(z)=\frac{1}{1+\exp{(-x)}}$ is called [standard logistic function](https://en.wikipedia.org/wiki/Logistic_regression#:~:text=Logistic%20regression%20is%20a%20statistical,a%20form%20of%20binary%20regression).

**Equivalent expression**:
  - Denote $p = P(y=1|\mathbf{x})$, then we can write in linear form
  $$\ln\frac{p}{1-p}=\tilde{x}\beta$$
  - Since $y$ only takes value in 0 or 1, we have
  $$P(y|\mathbf{x},\beta) = f(\mathbf{x};\beta)^y \big(1 - f(\mathbf{x};\beta) \big)^{1-y}$$
  
**MLE (Maximum Likelihood Estimation)**

Assume the samples are independent. The overall probibility to witness the whole training dataset
$$
{\begin{aligned}
&P(\mathbf{y}\; | \; \mathbf{X};\beta )\\
=&\prod _{i=1}^N P\left(y^{(i)}\mid \mathbf{x}^{(i)};\beta\right)\\
=&\prod_{i=1}^N f\big(\mathbf{x}^{(i)};\beta \big)^{y^{(i)}} 
\Big(1-f\big(\mathbf{x}^{(i)};\beta\big) \Big)^{\big(1-y^{(i)}\big)}
\end{aligned}}.
$$

Then we derive the loss function
$$
L (\beta) = L (\beta; X,\mathbf{y}) = - \frac{1}{N}\sum_{i=1}^N 
\Bigl\{y^{(i)} \ln\big( f(\mathbf{x}^{(i)};\beta) \big) 
+ (1 - y^{(i)}) \ln\big( 1 - f(\mathbf{x}^{(i)};\beta) \big) \Bigr\}.
$$

## Algorithm

Take the gradient (left as exercise -- if you like)
$$
\frac{\partial L (\beta)}{\partial \beta_{k}} =\frac{1}{N}\sum_{i=1}^N  \big(\sigma(\tilde{x}^{(i)}\beta)  - y^{(i)} \big) \tilde{x}^{(i)}_k.
$$

In vector form
$$
\nabla_{\beta} \big( L (\beta) \big) = \sum_{i=1}^N  \big(\sigma(\tilde{x}^{(i)})  - y^{(i)} \big) \tilde{x}^{(i)} 
=\frac{1}{N}\sum_{i=1}^N \big( f(\mathbf{x}^{(i)};\beta)  - y^{(i)} \big) \tilde{x}^{(i)}.
$$